In [1]:
import pandas as pd
from fuzzywuzzy import fuzz, process
import re
import numpy as np
import os

import time

## Setting names, files, and parameters
These can go into a dictionary _I believe_

In [2]:
country = "AR"
nq_date = "191018"

In [3]:
census = pd.read_csv("../../raw/ipums/"+country+"/ipumsi_00015.csv")

In [4]:
netquest = pd.read_csv("../../../Data/"+country+"/panel/"+country+"_netquest-panel.csv")

C:\Users\schadem\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (23,549) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
nq_dict = pd.read_excel("../../../Data/AR/panel/AR_levels.xlsx",
                       )

In [6]:
geo1_nq = "AR_provincia"
geo2_nq = "AR_departamento"
geo1_ipums = 'GEO1_AR2010'
geo2_ipums = 'GEO2_AR2010'
year = '2010'

In [7]:
ipums_geo2 = pd.read_csv('../../../Data/'+country+'/sample/ipums_codebook_'+geo2_ipums+'.csv',encoding='Latin1', names=['code','name'], skiprows=1)
ipums_geo1 = pd.read_csv('../../../Data/'+country+'/sample/ipums_codebook_'+geo1_ipums+'.csv',encoding='Latin1', names=['code','name'], skiprows=1)

Make sure Marcos Paz is here:

In [8]:
ipums_geo2[ipums_geo2.name.str.contains('Marcos')]

,code,name
64,6050,Marcos Paz
132,14010,Marcos Juárez


## Setting up helper structures

In [9]:
nq_geo1 = nq_dict[nq_dict.Variable==geo1_nq]
nq_geo1.columns = ["Variable",geo1_nq,geo1_nq+"_name"]
nq_geo1 = nq_geo1[[geo1_nq,geo1_nq+"_name"]]

nq_geo2 = nq_dict[nq_dict.Variable==geo2_nq]
nq_geo2.columns = ["Variable",geo2_nq,geo2_nq+"_name"]
nq_geo2 = nq_geo2[[geo2_nq,geo2_nq+"_name"]]
# nq_geo1 = pd.read_excel("../Data/AR/panel/netquest_codebook_AR_provincia.xlsx")

# nq_geo2 = pd.read_excel("../Data/AR/panel/netquest_codebook_AR_departamento.xlsx")

Setting up the unique DF for geographies for IPUMS

In [10]:
ipums_geodf = census[[geo1_ipums,geo2_ipums]]
ipums_geodf.columns = ['geo1_code','geo2_code']
ipums_geodf.drop_duplicates(subset=['geo1_code','geo2_code'],inplace=True)

ipums_geodf['geo1_name'] = ipums_geodf.merge(ipums_geo1, 
                                             how='left', 
                                             left_on = "geo1_code", 
                                             right_on = 'code', 
                                             copy=False)['name'].values

ipums_geodf['geo2_name'] = ipums_geodf.merge(ipums_geo2, 
                                             how='left', 
                                             left_on = "geo2_code", 
                                             right_on = 'code', 
                                             copy=False)['name'].values

C:\Users\schadem\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\schadem\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\schadem\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

We have to get the melting action going here.

In [11]:
ipums_geodf = ipums_geodf.geo2_name.str.split(',').apply(pd.Series) \
    .merge(ipums_geodf, right_index = True, left_index = True) \
    .drop(["geo2_name"], axis = 1) \
    .melt(id_vars = [k for k in ipums_geodf.columns if not (type(k)==int)|(k=='geo2_name')], value_name = "geo2_name") \
    .drop("variable", axis = 1) \
    .dropna(subset=['geo2_name'])
ipums_geodf.geo2_name = ipums_geodf.geo2_name.str.strip()

In [12]:
ipums_geodf = ipums_geodf.geo1_name.str.split(',').apply(pd.Series) \
    .merge(ipums_geodf, right_index = True, left_index = True) \
    .drop(["geo1_name"], axis = 1) \
    .melt(id_vars = [k for k in ipums_geodf.columns if not (type(k)==int)|(k=='geo1_name')], value_name = "geo1_name") \
    .drop("variable", axis = 1) \
    .dropna(subset=['geo1_name'])
ipums_geodf.geo1_name = ipums_geodf.geo1_name.str.strip()

## Recoding certain things
This could go to external functions to load in

Accounting for the CABA mess. Reducing to just one centroid for all of CABA.

In [13]:
ipums_geodf.loc[ipums_geodf.geo1_name=="City of Buenos Aires",
             'geo1_name'] = 'CABA'
ipums_geodf.loc[(ipums_geodf.geo1_name=="CABA"),'geo2_name'] = 'CABA'

In [14]:
caba_codes = ipums_geodf[ipums_geodf.geo1_name=="CABA"].geo2_code

Dropping CABA-related duplicates:

In [15]:
ipums_geodf.drop_duplicates(subset=["geo1_name","geo2_name"], inplace=True)

In [16]:
ipums_geodf.shape

(511, 4)

In [17]:
ipums_geodf[ipums_geodf.geo1_name=="CABA"]

,geo1_code,geo2_code,geo2_name,geo1_name
0,2,2006,CABA,CABA


Make sure all CABAs in the census table have that same leftover geo2_code:

In [18]:
ipums_geodf[ipums_geodf.geo1_name=="CABA"].geo2_code.values[0]

2006

In [19]:
census.loc[census[geo2_ipums].isin(caba_codes),geo2_ipums] = \
len(census.loc[census[geo2_ipums].isin(caba_codes),geo2_ipums])*[ipums_geodf[ipums_geodf.geo1_name=="CABA"].geo2_code[0]]

#= ipums_geodf[ipums_geodf.geo1_name=="CABA"].geo2_code.values

Now to Netquest--first, get the geo1- and geo2_name in there.

Import barrio as departamento in CABA--actually, that turns out not to be a great idea, since it seems that these are actually partidos of the province.

Instead, give these a unique code, that will be used to identify them later.

In [20]:
# netquest.loc[netquest[geo2_nq].isna()&netquest.AR_barrio.notna(),geo2_nq
#             ] = netquest.AR_barrio[netquest[geo2_nq].isna()&netquest.AR_barrio.notna()]

In [21]:
netquest.loc[netquest[geo2_nq].isna()&(netquest[geo1_nq]==1),geo2_nq] = 999

In [22]:
# province_labels = pd.read_excel("../Data/CL/provincia_dict.xlsx", encoding='Latin1', header=None)
nq_geodf = netquest.merge(nq_geo2, on=geo2_nq, how='left')\
                   .merge(nq_geo1, on=geo1_nq, how='left')[[geo1_nq,geo1_nq+'_name',geo2_nq,geo2_nq+'_name']]
nq_geodf.columns = ['geo1_code','geo1_name','geo2_code','geo2_name']
nq_geodf.drop_duplicates(inplace=True)

# Fixing CABA not having Dept.
nq_geodf.loc[nq_geodf.geo1_name=="Ciudad Autónoma de Buenos Aires",
             'geo1_name'] = 'CABA'
# Where province is CABA and no departamento exists, call these "CABA"
# nq_geodf.loc[(nq_geodf.geo1_name=="CABA")&(nq_geodf.geo2_name.isna()),'geo2_name'] = 'CABA'

nq_geodf = nq_geodf[(nq_geodf['geo1_code'].notna()) & (nq_geodf['geo2_code'].notna())] 

In [23]:
nq_geodf[nq_geodf.geo1_name=="CABA"]
nq_geodf[nq_geodf.geo1_code==1]

,geo1_code,geo1_name,geo2_code,geo2_name
11,1.0,CABA,999.0,NaN
32832,1.0,CABA,133.0,Villarino
104299,1.0,CABA,156.0,Sobremonte
118839,1.0,CABA,21.0,Cañuelas
155728,1.0,CABA,2.0,9 de Julio
168624,1.0,CABA,67.0,La Matanza
188767,1.0,CABA,8.0,Avellaneda
236989,1.0,CABA,1.0,25 de Mayo


In [24]:
nq_geodf[nq_geodf.duplicated('geo2_code', keep=False)].sort_values('geo2_code')

,geo1_code,geo1_name,geo2_code,geo2_name
204,2.0,Buenos Aires,1.0,25 de Mayo
236989,1.0,CABA,1.0,25 de Mayo
531,2.0,Buenos Aires,2.0,9 de Julio
155728,1.0,CABA,2.0,9 de Julio
36,2.0,Buenos Aires,8.0,Avellaneda
188767,1.0,CABA,8.0,Avellaneda
83,2.0,Buenos Aires,21.0,Cañuelas
118839,1.0,CABA,21.0,Cañuelas
15,2.0,Buenos Aires,67.0,La Matanza
168624,1.0,CABA,67.0,La Matanza


## removing duplicates

Finding the duplicates in nq_geodf.geo2_code, and keeping the combinations that occur most often (which are quite clearly the good ones).

In [25]:
nq_geodf['count'] = nq_geodf.apply(
    lambda r: sum((netquest[geo1_nq]==r['geo1_code'])&
                  (netquest[geo2_nq]==r['geo2_code'])),
                                   axis=1)
# nq_geodf.sort_values(['geo2_code','count']) \
#     [(nq_geodf.sort_values(['geo2_code','count']).duplicated('geo2_code',keep=False))]
nq_geodf = nq_geodf.sort_values(['geo2_code','count']) \
    [~(nq_geodf.sort_values(['geo2_code','count']).duplicated('geo2_code',keep='last'))]

In [26]:
nq_geodf.shape

(507, 5)

In [27]:
ipums_geodf[ipums_geodf.geo1_name.str.contains("CABA")]

,geo1_code,geo2_code,geo2_name,geo1_name
0,2,2006,CABA,CABA


In [28]:
nq_geodf[nq_geodf.geo1_name.str.contains("CABA")]

,geo1_code,geo1_name,geo2_code,geo2_name,count
11,1.0,CABA,999.0,NaN,30475


## a function for fuzzy-joining columns from dataframes

In [29]:
def fuzzy_join(df1, df2, varname, subset=False, cutoff=90):
    '''Returns a dataframe the length of df1, with the matches on the given variable
    
        Assumes the var is named the same in both dataframes.
        
        subset is a tuple or list of length 2 whose first element is a string indicating which
        variable in df1 has to be equal to which variable in df2 (specified by 2nd element)
        '''
    #figure out if there is a constraint
    if not subset:
        #match #1
        ratio_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname].astype(str).drop_duplicates(),
                                             scorer=fuzz.ratio, limit=2
                                            ), axis=1)
        #match #2
        parti_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname].astype(str).drop_duplicates(),
                                             scorer=fuzz.partial_ratio, limit=2
                                            ), axis=1)
        
    else:
        #match #1
        ratio_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname][df2[subset[1]]==d[subset[0]]].astype(str).drop_duplicates(),
                                             scorer=fuzz.ratio, limit=2
                                            ), axis=1)
        #match #2
        parti_matches = df1.astype(str).apply(
                lambda d: process.extract(d[varname], 
                                             df2[varname][df2[subset[1]]==d[subset[0]]].astype(str).drop_duplicates(),
                                             scorer=fuzz.partial_ratio, limit=2
                                            ), axis=1)    

    
    # different match cases
    #morRatioMatch = ratio_matches.apply(lambda l: (l[0][1]==100)&(l[1][1]==100))
    oneRatioMatch = ratio_matches.apply(lambda l: (l[0][1]>=cutoff))#&(l[1][1]<100))
    noRatioMatch = ratio_matches.apply(lambda l: (l[0][1]<cutoff))

    #morPartiMatch = parti_matches.apply(lambda l: (l[0][1]==100)&(l[1][1]==100))
    onePartiMatch = parti_matches.apply(lambda l: (l[0][1]>=cutoff))#&(l[1][1]<100))
    noPartiMatch = parti_matches.apply(lambda l: (l[0][1]<cutoff))
    
    # pick out what's better
    matches = pd.Series([(np.nan,np.nan,np.nan)]*len(df1),index=df1.index)
    matches.loc[oneRatioMatch] = ratio_matches.loc[oneRatioMatch].apply(lambda l: l[0])
    matches.loc[(noRatioMatch&onePartiMatch)] = parti_matches.loc[(noRatioMatch&onePartiMatch)]\
        .apply(lambda l: l[0])
    matches.loc[(noRatioMatch&noPartiMatch)] = parti_matches.loc[(noRatioMatch&noPartiMatch)]\
        .apply(lambda l: l[0])
    
    df = pd.DataFrame()
    df['name'] = matches.apply(lambda l: l[0])
    df['score'] = matches.apply(lambda l: l[1])
    df['index'] = matches.apply(lambda l: l[2])
    df['parti_matches'] = parti_matches
    df['ratio_matches'] = ratio_matches
    
    return df

Use the new tool!

## Manual fixes, as found necessary in the next step...
Might want to outsource these into external function

In [30]:
nq_geodf.loc[nq_geodf.geo2_code==999, 'geo2_name'] = "CABA"

ipums_geodf.loc[ipums_geodf.geo1_name.str.contains("Buenos Aires province"),
             'geo1_name'] = "Buenos Aires"

ipums_geodf.loc[ipums_geodf.geo2_name.str.contains("Puan"),
                'geo2_name'] = "Puán"
ipums_geodf.loc[(ipums_geodf.geo2_name.str.contains("General San Martín"))\
                &(ipums_geodf.geo1_name=="Buenos Aires"),
                'geo2_name'] = "Ciudad Libertador San Martín"
ipums_geodf.loc[(ipums_geodf.geo2_name.str.contains("La Capital"))\
                &(ipums_geodf.geo1_name=="San Luis"),
                'geo2_name'] = "Juan Martín de Pueyrredón"
ipums_geodf.loc[(ipums_geodf.geo2_name=="Maipú")\
                &(ipums_geodf.geo2_code==6050),
                'geo2_name'] = "Marcos Paz"



ipums_geodf.loc[(ipums_geodf.geo2_name.str.contains("Chascomus")),
                'geo2_name'] = "Chascomús"
ipums_geodf.loc[(ipums_geodf.geo2_name.str.contains("Jose C. Paz")),
                'geo2_name'] = "José C. Paz"
# nq_geodf.loc[(nq_geodf.geo2_name.str.contains("Adolfo Gonzales Chaves")),
#                 'geo2_name'] = "Adolfo González Chaves"
nq_geodf.loc[(nq_geodf.geo2_name.str.contains("Paso de Indios"))\
                &(nq_geodf.geo1_name=="Chubut"),
                'geo2_name'] = "Paso de los Indios"
nq_geodf.loc[(nq_geodf.geo2_name.str.contains("Coronel de Marina Leonardo Rosales"))\
                &(nq_geodf.geo1_name=="Buenos Aires"),
                'geo2_name'] = "Coronel de Marine L. Rosales"
ipums_geodf.loc[(ipums_geodf.geo2_name.str.contains("Veinticinco de Mayo"))\
                &(ipums_geodf.geo1_name=="Buenos Aires"),
                'geo2_name'] = "25 de Mayo"
nq_geodf.loc[(nq_geodf.geo2_name.str.contains("Pueyrredón"))\
                &(nq_geodf.geo1_name=="Buenos Aires"),
                'geo2_name'] = "General Pueyrredón"

## Fuzzy joining

In [33]:
nq_geodf[['geo1_match_name','geo1_match_score','geo1_match_index']] \
= fuzzy_join(nq_geodf, ipums_geodf, 'geo1_name')[['name','score','index']]

In [34]:
nq_geodf[['geo2_match_name','geo2_match_score','geo2_match_index']] \
= fuzzy_join(nq_geodf, ipums_geodf, 'geo2_name', 
             subset=['geo1_match_name','geo1_name']
            )[['name','score','index']]

## Double-check

In [35]:
tmp = nq_geodf.copy()

**What is still an issue??**

In [36]:
tmp[tmp.duplicated('geo2_code', keep=False)].sort_values('geo2_code')\
[["geo1_code","geo2_code",
  "geo1_name","geo1_match_name",
  "geo2_name","geo2_match_name",
  "count"]]

,geo1_code,geo2_code,geo1_name,geo1_match_name,geo2_name,geo2_match_name,count


Success! This can be the geo-df.

In [37]:
nq_geodf = tmp

In [38]:
nq_geodf[(nq_geodf.geo2_match_score<71)][['geo1_name',
                                          'geo1_code',
#                                         'geo1_match_name',
                                        'geo2_name',
                                        'geo2_code',
                                        'geo2_match_name',
                                        'geo2_match_score'
                                       ]
                                      ].sort_values('geo1_name'
                                      )#.to_csv("./faulty_matches.csv")

,geo1_name,geo1_code,geo2_name,geo2_code,geo2_match_name,geo2_match_score


In [39]:
ipums_geodf[["geo1_name","geo2_name","geo2_code"]
           ][ipums_geodf.geo2_name.str.contains("Marcos Paz")]

,geo1_name,geo2_name,geo2_code
80,Buenos Aires,Marcos Paz,6050


## This currently requires manual intervention

In [40]:
has_ipums_geo = nq_geodf.geo2_match_score>70 #&nq_geodf.geo1_match_index.notna()

In [41]:
sum(has_ipums_geo)
len(nq_geodf)

507

## Attaching code

In [42]:
nq_geodf['IPUMS_geo2_code'] = np.nan

nq_geodf.loc[has_ipums_geo,'IPUMS_geo2_code'] = nq_geodf[has_ipums_geo]\
                            .geo2_match_index\
                            .astype(int)\
                            .apply(
                                lambda i: ipums_geodf.loc[i,'geo2_code']
                            .astype(int)
)

We want to replace each nq_geo2 with the _code_ for the corresponding census geography.



Read in the municipio-centroids, and hopefully attach them here.

In [43]:
geo2_centroids = pd.read_csv('../../../Data/'+country+'/geography/'+country+'_geo2_centroids.csv', encoding='latin1')

Merge centroids onto NQ geometries:

In [44]:
nq_geodf_merged = nq_geodf.merge(geo2_centroids[['ADMIN_NAME','Y','X','IPUM'+year]], 
               left_on='IPUMS_geo2_code',
               right_on="IPUM"+year,
               how='left'
              ).drop('IPUM'+year, axis=1)

Merge NQ geometries onto NQ data:

In [46]:
panel_geo = netquest.merge(nq_geodf_merged[['X','Y','geo2_code']],
               left_on=geo2_nq,
               right_on='geo2_code',
               how='left'
              )

## Writing out
Filenames should be stored externally for each country

In [47]:
panel_geo.shape[0]==netquest.shape[0]

True

In [48]:
panel_geo.to_csv('../../../Data/'+country+'/panel/'+country+'_netquest-panel_geo.csv')

And the same for census: attach geographies

In [49]:
census_geo = census.merge(geo2_centroids[['ADMIN_NAME','X',"Y",'IPUM'+year]],
                          left_on = geo2_ipums,
                          right_on='IPUM'+year,
                          how='left'
                         ).drop('IPUM'+year,axis=1)

In [50]:
census_geo.shape[0]==census.shape[0]

True

In [53]:
census_geo.to_csv('../../../Data/'+country+'/sample/'+country+'_ipums-census_geo.csv')